In [1]:
# Get raw data
with open('input/05.txt', 'r') as f:
    rawinput = f.read().strip()

In [2]:
# Part 1
class Program(object):
    def __init__(self, instr, input_val):
        self.ops = {
            1: self.do_add,
            2: self.do_mult,
            3: self.do_input,
            4: self.do_output,
            }
        self.instr_orig = instr
        self.input_val = input_val
        self.reset()
        
    def reset(self):
        self.instr = [*self.instr_orig]
        self.ptr = 0
        self.output = []
        
    def actuals(self, params):
        return [self.instr[i] if j=='0' else i 
                for i,j in params]
        
    def do_add(self, params):
        self.instr[params[2][0]] = sum(self.actuals(params[:2]))
        self.ptr += 4

    def do_mult(self, params):
        self.instr[params[2][0]] = (z:=self.actuals(params[:2]))[0]*z[1]
        self.ptr += 4
        
    def do_input(self, params):
        self.instr[params[0][0]] = self.input_val
        self.ptr += 2
        
    def do_output(self, params):
        self.output += self.actuals(params[:1])
        self.ptr += 2
        
    def do_exec(self):
        self.reset()
        while True:
            opmode = self.instr[self.ptr]
            opcode = opmode % 100
            if opcode == 99:
                break
            params = [[i,j]
                      for i,j in zip(self.instr[self.ptr+1:], 
                                     list((str(opmode)[-3::-1]+'000'))[:3])]
            self.ops[opcode](params)

program = Program([int(i) for i in rawinput.split(',')], 1)
program.do_exec()
assert(all([i==0 for i in program.output[:-1]]))
program.output[-1]

9961446

In [3]:
# Part 2
class ProgramV2(Program):
    def __init__(self, instr, input_val):
        self.ops = {
            1: self.do_add,
            2: self.do_mult,
            3: self.do_input,
            4: self.do_output,
            5: self.do_jump_if_true,
            6: self.do_jump_if_false,
            7: self.do_less_than,
            8: self.do_equals,
            }
        self.instr_orig = instr
        self.input_val = input_val
        self.reset()

    def do_jump_if_true(self, params):
        if (z:=self.actuals(params[:2]))[0]:
            self.ptr = z[1]
        else:
            self.ptr += 3

    def do_jump_if_false(self, params):
        if (z:=self.actuals(params[:2]))[0] == 0:
            self.ptr = z[1]
        else:
            self.ptr += 3

    def do_less_than(self, params):
        self.instr[params[2][0]] = int((z:=self.actuals(params[:2]))[0] < z[1])
        self.ptr += 4
        
    def do_equals(self, params):
        self.instr[params[2][0]] = int((z:=self.actuals(params[:2]))[0] == z[1])
        self.ptr += 4
        
program = ProgramV2([int(i) for i in rawinput.split(',')], 5)
program.do_exec()
program.output[-1]

742621